In [1]:
import os
import setGPU
import numpy as np
import h5py
from matplotlib import pyplot as plt
import matplotlib
import pandas as pd
from decimal import Decimal
import torch 
from sklearn.preprocessing import StandardScaler
from torch.utils.data import Dataset, DataLoader
from glob import glob
import torch
from torch.autograd import Variable
import time
import torch.nn as nn
import torch.nn.parallel
import torch.backends.cudnn as cudnn
import torch.distributed as dist
import torch.optim
import torch.utils.data.distributed
import torch.nn.functional as F
from torch.utils.data.sampler import SubsetRandomSampler
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from operator import itemgetter
import itertools

setGPU: Setting GPU to: 0


/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
N=188
infile = h5py.File('/bigdata/shared/HLS4ML/jetImage.h5', 'r')
sorted_pt_constituents = np.array(infile.get('jetConstituentList'))
scaled_jets = infile.get('jets') # in any case
mass_targets = scaled_jets[:,-6:-1]
print("Loading completed")

Loading completed


In [3]:
import torch.nn.functional as F
class GraphNet(nn.Module):
    def __init__(self, n_constituents, n_targets, params, hidden):
        super(GraphNet, self).__init__()
        self.hidden = hidden
        self.P = params
        self.N = n_constituents
        self.Nr = self.N * (self.N - 1)
        self.Dr = 0
        self.De = 5
        self.Dx = 0
        self.Do = 6
        self.n_targets = n_targets
        self.assign_matrices()
        self.Ra = Variable(torch.ones(self.Dr, self.Nr))
        self.fr1 = nn.Linear(2 * self.P + self.Dr, hidden)
        self.fr2 = nn.Linear(hidden, int(hidden/2))
        self.fr3 = nn.Linear(int(hidden/2), self.De)
        self.fo1 = nn.Linear(self.P + self.Dx + self.De, hidden)
        self.fo2 = nn.Linear(hidden, int(hidden/2))
        self.fo3 = nn.Linear(int(hidden/2), self.Do)
        self.fc1 = nn.Linear(400, hidden)
        self.fc2 = nn.Linear(hidden, int(hidden/2))
        self.fc3 = nn.Linear(int(hidden/2), self.n_targets)
        self.avg = nn.AdaptiveAvgPool1d(200)
        self.mp = nn.AdaptiveMaxPool1d(200)
    def assign_matrices(self):
        self.Rr = torch.zeros(self.N, self.Nr)
        self.Rs = torch.zeros(self.N, self.Nr)
        receiver_sender_list = [i for i in itertools.product(range(self.N), range(self.N)) if i[0]!=i[1]]
        for i, (r, s) in enumerate(receiver_sender_list):
            self.Rr[r, i] = 1
            self.Rs[s, i] = 1
        self.Rr = Variable(self.Rr).cuda()
        self.Rs = Variable(self.Rs).cuda()
        
    def forward(self, x):
        x=torch.transpose(x, 1, 2).contiguous()
        Orr = self.tmul(x, self.Rr)
        Ors = self.tmul(x, self.Rs)
#         print("This is the Orr {}". format(Orr.shape))
#         print("This is the Ors {}". format(Ors.shape))
        B = torch.cat([Orr, Ors], 1)
#         print("This is what after concat, actual B is {}". format(B.shape))
        ### First MLP ###
        B = torch.transpose(B, 1, 2).contiguous()
#         print("size of B after transposed is {}". format(B.shape))
        B = nn.functional.relu(self.fr1(B.view(-1, 2 * self.P + self.Dr)))
        B = nn.functional.relu(self.fr2(B))
        E = nn.functional.relu(self.fr3(B).view(-1, self.Nr, self.De))
#         print("size of E {}". format(E.shape))
        del B
        E = torch.transpose(E, 1, 2).contiguous()
#         print("size of E after transposed{}". format(E.shape))
        Ebar = self.tmul(E, torch.transpose(self.Rr, 0, 1).contiguous())
        del E
        C = torch.cat([x, Ebar], 1)
        C = torch.transpose(C, 1, 2).contiguous()
        ### Second MLP ###
        C = nn.functional.relu(self.fo1(C.view(-1, self.P + self.Dx + self.De)))
        C = nn.functional.relu(self.fo2(C))
        O = nn.functional.relu(self.fo3(C).view(-1, self.N, self.Do))
        del C
        ### Classification MLP ###
        batch_size,max_len, features = O.size()
        O = O.contiguous().view(batch_size, -1).unsqueeze(0)
        N = self.avg(O).squeeze(0)
        N2 = self.mp(O).squeeze(0)
        N = torch.cat((N,N2),1)
        N = nn.functional.relu(self.fc1(N))
        del O
        N = nn.functional.relu(self.fc2(N))
        N = self.fc3(N)
        return N
    def tmul(self, x, y):  #Takes (I * J * K)(K * L) -> I * J * L 
        x_shape = x.size()
        y_shape = y.size()
#         print(x_shape)
#         print(y_shape)
        return torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])

    
model = GraphNet(n_constituents=188,n_targets=5,params=3, hidden=10)
model.cuda()
print(model)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
#criterion = nn.NLLLoss()
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer,mode ='min',factor=0.5,patience=10)

GraphNet(
  (fr1): Linear(in_features=6, out_features=10, bias=True)
  (fr2): Linear(in_features=10, out_features=5, bias=True)
  (fr3): Linear(in_features=5, out_features=5, bias=True)
  (fo1): Linear(in_features=8, out_features=10, bias=True)
  (fo2): Linear(in_features=10, out_features=5, bias=True)
  (fo3): Linear(in_features=5, out_features=6, bias=True)
  (fc1): Linear(in_features=400, out_features=10, bias=True)
  (fc2): Linear(in_features=10, out_features=5, bias=True)
  (fc3): Linear(in_features=5, out_features=5, bias=True)
  (avg): AdaptiveAvgPool1d(output_size=200)
  (mp): AdaptiveMaxPool1d(output_size=200)
)


In [4]:
#/////////////for later usage///////
# def get_sample(training, target, choice):
#     target_vals = np.argmax(target, axis = 1)
#     ind, = np.where(target_vals == choice)
#     chosen_ind = np.random.choice(ind, 50000)
#     return training[chosen_ind], target[chosen_ind]

# n_targets = target.shape[1]
# samples = [get_sample(training, target, i) for i in range(n_targets)]
# trainings = [i[0] for i in samples]
# targets = [i[1] for i in samples]
# big_training = np.concatenate(trainings)
# big_target = np.concatenate(targets)
# big_training, big_target = util.shuffle_together(big_training, big_target)

In [5]:
class EventDataset(Dataset):
    def __init__(self, constituents, targets,
                 constituents_name = ['j1_pt', 'j1_etarel','j1_phirel']
                ):
        self.constituents = torch.from_numpy(constituents)
        self.targets = torch.from_numpy(targets)
        self.constituents_name = constituents_name
    def __len__(self):
        return self.constituents.shape[0]
    def __getitem__(self,idx):
        return self.constituents[idx], self.targets[idx]

In [6]:
test_size = 0.2
valid_size = 0.25
batch_size = 128
pin_memory = False
num_workers = 4
epochs = 200

In [7]:
def shuffle(a,b):
    iX = a.shape[1]
    iY = a.shape[2]
    b_shape = b.shape[1]
    a = a.reshape(a.shape[0], iX*iY)
    total = np.column_stack((a,b))
    np.random.shuffle(total)
    a = total[:,:iX*iY]
    b = total[:,iX*iY:iX*iY+b_shape]
    a = a.reshape(a.shape[0],iX, iY)
    return a,b

In [8]:
sorted_pt_constituents, mass_targets = shuffle(sorted_pt_constituents, mass_targets)

In [9]:
#for commenting
sorted_pt_constituentsnp= sorted_pt_constituents

num_train = sorted_pt_constituentsnp.shape[0]
split = int(np.floor(test_size * num_train))
#for commenting
sorted_indices = list(range(num_train))
split_v=int(np.floor(valid_size * (num_train-split)))
print(split)
print(split_v)
train_idx, test_idx = sorted_indices[split:], sorted_indices[:split]
train_idx, valid_idx = train_idx[split_v:], train_idx[:split_v]


training_data = sorted_pt_constituentsnp[train_idx, :]
validation_data = sorted_pt_constituentsnp[valid_idx, :]
testing_data = sorted_pt_constituentsnp[test_idx,:]
y_train = mass_targets[train_idx, :]
y_valid = mass_targets[valid_idx, :]
y_test = mass_targets[test_idx, :]


# iSplit_1 = int(0.6*mass_targets.shape[0])
# iSplit_2 = int(0.8*mass_targets.shape[0])

# training_data= sorted_pt_constituents[:iSplit_1, :]
# validation_data = sorted_pt_constituents[iSplit_1:iSplit_2, :]
# testing_data = sorted_pt_constituents[iSplit_2:, :]

19742
19742


In [10]:
# y_train = mass_targets[:iSplit_1, :]
# y_valid = mass_targets[iSplit_1:iSplit_2, :]
# y_test = mass_targets[iSplit_2:, :]

y_train_1D =np.argmax(y_train, axis=1)
y_valid_1D = np.argmax(y_valid, axis=1)
y_test_1D = np.argmax(y_test, axis=1)
# print(y_train_1D.shape)
# print(y_train_1D[:10])
train = EventDataset(training_data,y_train_1D)  
valid = EventDataset(validation_data,y_valid_1D)  


#get the seq length
# t_seq_length= [ arr[i] for i in train_idx]
# v_seq_length= [ arr[i] for i in valid_idx]
# ts_seq_length=[ arr[i] for i in test_idx]

# print(len(t_seq_length[0:1024]))
# print(train_idx[:10])
# print(arr[train_idx[:10]])

print("Load dataset")
train_loader = DataLoader(dataset = train, batch_size = batch_size, shuffle = True, num_workers = 4)
valid_loader = DataLoader(dataset = valid, batch_size = batch_size, shuffle = True, num_workers = 4) 
data_loader = {"training" :train_loader, "validation" : valid_loader} 
print("Finished")
####### finished 

Load dataset
Finished


In [11]:
def train_model(num_epochs, model, criterion, optimizer,scheduler,volatile=False):

    best_model = model.state_dict()
    best_acc = 0.0
    train_losses ,val_losses = [],[]
    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('_' * 10)

        # Each epoch has a training and validation phase
        for phase in ['training', 'validation']:
            if phase == 'training':
                model.train() # Set model to training mode
            else:
                model.eval() # Set model to evaluate mode
                volatile=True
            running_loss = 0.0
            running_corrects = 0

        # Iterate over data.
            for batch_idx, (x_data, y_data) in enumerate(data_loader[phase]):
                x_data, y_data = Variable(x_data.cuda().type(torch.cuda.FloatTensor),volatile),Variable(y_data.cuda().type(torch.cuda.LongTensor))
#                 beg = batch_size*batch_idx
#                 end = min((batch_idx+1)*batch_size, split)
#                 print("Beg is %d" % beg)
#                 print("Beg is %d" % end)
#                 if phase == 'training':
#                     x_data = torch.nn.utils.rnn.pack_padded_sequence(x_data, t_seq_length[beg:end], batch_first=True)
#                 else:
#                     x_data = torch.nn.utils.rnn.pack_padded_sequence(x_data, v_seq_length[beg:end], batch_first=True)
                if phase == 'training':
                    optimizer.zero_grad()
                # forwardgyg
                outputs = model(x_data)
                _, preds = torch.max(outputs.data, 1)
                loss = criterion(outputs, y_data)
                # backward + optimize only if in training phase
                if phase == 'training':
                    loss.backward()
                    optimizer.step()
                
                # statistics
                running_loss += loss.data[0]
                running_corrects += torch.sum(preds == y_data.data)
                #print("I finished %d batch" % batch_idx)
            
            epoch_loss = running_loss / len(data_loader[phase].dataset)
            epoch_acc = 100. * running_corrects / len(data_loader[phase].dataset)
            if phase == 'training':
                train_losses.append(epoch_loss)
            else:
                scheduler.step(epoch_loss)
                val_losses.append(epoch_loss)
            
            print('{} Loss: {:.4f} Acc: {:.4f}'.format(
                phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'validation' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model = model.state_dict()

            print()

    
    print('Best val Acc: {:4f}'.format(best_acc))
    # load best model weights
    model.load_state_dict(best_model)
    return model,train_losses ,val_losses

In [ ]:
model_output,train_losses,val_losses = train_model(epochs,model,criterion,optimizer,scheduler)

Epoch 0/199
__________
training Loss: 0.0110 Acc: 33.9226

validation Loss: 0.0104 Acc: 39.0690

Epoch 1/199
__________
training Loss: 0.0101 Acc: 40.4333

validation Loss: 0.0100 Acc: 41.3585

Epoch 2/199
__________
training Loss: 0.0099 Acc: 41.9579

validation Loss: 0.0099 Acc: 42.1538

Epoch 3/199
__________
training Loss: 0.0099 Acc: 42.4189

validation Loss: 0.0099 Acc: 41.9461

Epoch 4/199
__________
training Loss: 0.0098 Acc: 42.5067

validation Loss: 0.0099 Acc: 42.5438

Epoch 5/199
__________
training Loss: 0.0098 Acc: 43.0622

validation Loss: 0.0099 Acc: 43.0909

Epoch 6/199
__________
training Loss: 0.0097 Acc: 44.1056

validation Loss: 0.0097 Acc: 44.4838

Epoch 7/199
__________
training Loss: 0.0095 Acc: 46.0018

validation Loss: 0.0094 Acc: 47.2039

Epoch 8/199
__________
training Loss: 0.0093 Acc: 48.0178

validation Loss: 0.0094 Acc: 48.0448

Epoch 9/199
__________
training Loss: 0.0092 Acc: 48.9903

validation Loss: 0.0091 Acc: 48.8907

Epoch 10/199
__________
traini

In [ ]:
plt.figure()
plt.plot(train_losses)
plt.plot(val_losses)
plt.plot()
plt.yscale('log')
plt.title('Interaction Network')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'valid'], loc='upper left')
plt.show()

In [ ]:
testing_data_tens=Variable(torch.from_numpy(testing_data),volatile=True).float().cuda()
#packed_cons_test = torch.nn.utils.rnn.pack_padded_sequence(testing_data_tens, ts_seq_length, batch_first=True)
labels = ['j_g', 'j_q', 'j_w', 'j_z', 'j_t']
labels_val = y_test 

In [ ]:
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
df = pd.DataFrame()
fpr = {}
tpr = {}
auc1 = {}
predict_test = model_output(testing_data_tens)
predict_test= predict_test.data.cpu().numpy()
plt.figure()
for i, label in enumerate(labels):
        df[label] = labels_val[:,i]
        df[label + '_pred'] = predict_test[:,i]

        fpr[label], tpr[label], threshold = roc_curve(df[label],df[label+'_pred'])

        auc1[label] = auc(fpr[label], tpr[label])

        plt.plot(tpr[label],fpr[label],label='%s tagger, auc = %.1f%%'%(label,auc1[label]*100.))
df.to_csv("GRU_without_pack_3.csv", sep='\t')
plt.semilogy()
plt.xlabel("sig. efficiency")
plt.ylabel("bkg. mistag rate")
plt.ylim(0.0001,1)
plt.grid(True)
plt.legend(loc='upper left')
#plt.savefig('%s/ROC.pdf'%(options.outputDir))
plt.show()

In [ ]:
# import itertools
# N=3
# receiver_sender_list = [i for i in itertools.product(range(N), range(N)) if i[0]!=i[1]]
# print(receiver_sender_list)
# Nr= N*(N-1)
# Rr = torch.zeros(N, Nr)
# Rs = torch.zeros(N, Nr)
# receiver_sender_list = [i for i in itertools.product(range(N), range(N)) if i[0]!=i[1]]
# for i, (r, s) in enumerate(receiver_sender_list):
#     Rr[r, i] = 1
#     Rs[s, i] = 1
# print(Rr)
# print(Rs)

In [ ]:
# x_shape = Rr.size()
# y_shape = Rs.size()
# x=Rr
# y=Rs
# print(x_shape)
# print(y_shape)
# torch.mm(x.view(-1, x_shape[2]), y).view(-1, x_shape[1], y_shape[1])      